# Basics for Spark using python
In this notebook we define how to use spark creating a session, changing the schema and selecting and adding data

In [0]:
# Do the imports
from pyspark.sql import SparkSession

In [0]:
# Create a session
spark =SparkSession.builder.appName('Basics').getOrCreate()

In [0]:
# upload the json file to DBFS
df = spark.read.json("/FileStore/tables/people.json")
df.show()
df.printSchema()
print(df.columns)
df.describe().show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

['age', 'name']
+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [0]:
# import structs so we can change schema
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [0]:
# define schema
data_schema = [StructField('age', IntegerType(), True),
              StructField('name', StringType(), True)]
final_struct = StructType(fields= data_schema)

In [0]:
# read json with new schema
df = spark.read.json("/FileStore/tables/people.json", final_struct)
df.printSchema()
df.describe().show()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [0]:
# types of selections
print(type(df['age']))
print(type(df.select('age')))
print(type(df.head(2)[0]))

#selecting columns
df.select(['age', 'name']).show()

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.types.Row'>
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
# adding column (not inplace)
df.withColumn('double_age', df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [0]:
# rename column (not inplace)
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [0]:
df.createOrReplaceTempView('people')
results = spark.sql("SELECT * FROM people")
results.show()
new_results = spark.sql("SELECT * FROM people WHERE age = 30")
new_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

